In [ ]:
%load_ext autoreload
%autoreload 2

import rospy
from rospy_tutorials.srv import AddTwoInts, AddTwoIntsResponse
import jupyros
from turtlesim.srv import Spawn

In [ ]:
rospy.init_node("adder")

In [ ]:
def handle_add_two_ints(req):
    print("Returning [%s + %s = %s]"%(req.a, req.b, (req.a + req.b)))
    return AddTwoIntsResponse(req.a + req.b)

In [ ]:
%%thread_cell

srv = rospy.Service('add_two_ints', AddTwoInts, handle_add_two_ints)

In [ ]:
jupyros.client('add_two_ints', AddTwoInts)

In [ ]:
# !rosrun turtlesim turtlesim_node

In [ ]:
jupyros.client('spawn', Spawn)